# Pension Fund Optimization Using Linear Programming (LP) – Armco Inc. Case Study
-- adapted by Prof Cassidey from the book *Practical Management Science*
## Base Case
James Judson is the financial manager in charge of the company pension fund at Armco Incorporated. James knows that the fund must be sufficient to make the payments listed in Table 4.4. Each payment must be made on the first day of each year. James is going to finance these payments by purchasing bonds. It is currently January 1 of year 1, and three bonds are available for immediate purchase. The prices and coupons for the bonds are as follows. (We assume that all coupon payments are received on January 1 and arrive in time to meet cash demands for the year on which they arrive.)

* Bond 1 costs $980 and yields a $60 coupon in the years 2 through 5 and a $1060 payment on maturity in the year 6.

* Bond 2 costs $970 and yields a $65 coupon in the years 2 through 11 and a $1065 payment on maturity in the year 12.

* Bond 3 costs $1050 and yields a $75 coupon in the years 2 through 14 and a $1075 payment on maturity in the year 15.

James must decide how much cash to allocate (from company coffers) to meet the initial $11,000 payment and buy enough bonds to make future payments. He knows that any excess cash on hand can earn an annual rate of 4% in a fixed-rate account. How should he proceed?

Develop an LP model that relates initial allocation of money and bond purchases to future cash availabilities, and to minimize the initialize allocation of money required to meet all future pension fund payments.  

In [14]:
import gurobipy as gp
from gurobipy import GRB
from itertools import product
import numpy as np

def get_solution(m):
    print("Solution Values:")
    for var in m.getVars():
        print(f"    {var.VarName}: {var.X}")

def get_shadow(m):
    print(f"Shadow Price Information:")
    for constraint in m.getConstrs():
        print(f"    {constraint.ConstrName}: {constraint.Pi}, (RHS = {constraint.RHS}), (LB = {round(constraint.SARHSLow, 2)}), (UB = {round(constraint.SARHSUp, 2)})")

def get_rc(m):
    print(f"Reduced Cost Information")
    for var in m.getVars():
        print(f"    {var.VarName}: {var.RC}, (Coef LB = {round(var.SAObjLow, 2)}), (Coef UB = {round(var.SAObjUp, 2)})")

## Parameters

In [15]:
#Input
T = 15 # length of the time horizon
B = [0, 1, 2] # bonds
COSTS = [980, 970, 1050] # costs in year one
D = [6, 12, 15] # duration (year of payout)

BASE_INCOMES = [60, 65, 75] # incomes in years 2 through (D_B - 1)
INCOMES = np.zeros((len(B), T))
PAYOUTS = [1060, 1065, 1075]

for b in B:
    for t in range(1, D[b] - 1):
        INCOMES[b, t] = BASE_INCOMES[b]
    INCOMES[b, D[b] - 1] = PAYOUTS[b]
    print(f"Bond {b} Incomes: {[x for x in INCOMES[b]]}")

BASE_r = .04

# Base Case
BASE_PAYMENTS = [x*1000 for x in [11, 12, 14, 15, 16, 18, 20, 21, 22, 24, 25, 30, 31, 31, 31]]
BASE_CASE = {
    'r': [BASE_r] * T, # interest rates are level
    'c' : BASE_PAYMENTS # payments go up steadily
}

# Bad Case
BAD_PAYMENTS = [11, 12, 14, 20, 20, 22, 23, 24, 26, 30, 31, 31, 31, 31, 31]
BAD_CASE = {
    'r': [.04, .04, .04, .03, .01, .0, .0, .0, .0, .0, .0, .01, .02, .03, .04], # interest rates start dropping in year 4, then stay at zero for many years before rising again
    'c': [x*1000 for x in BAD_PAYMENTS] # payments increase dramatically in year 4
}

Bond 0 Incomes: [0.0, 60.0, 60.0, 60.0, 60.0, 1060.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Bond 1 Incomes: [0.0, 65.0, 65.0, 65.0, 65.0, 65.0, 65.0, 65.0, 65.0, 65.0, 65.0, 1065.0, 0.0, 0.0, 0.0]
Bond 2 Incomes: [0.0, 75.0, 75.0, 75.0, 75.0, 75.0, 75.0, 75.0, 75.0, 75.0, 75.0, 75.0, 75.0, 75.0, 1075.0]


##  Base Case Model

In [16]:
payments = BASE_CASE['c']
# interest_rates = BASE_CASE['r']

# Create a new model
m = gp.Model('base_case')

# Create variables - Decision Variables
I = m.addVars(B, vtype=GRB.CONTINUOUS, name="I", lb=0.0) # Investment in each bond (shares to buy)
C = m.addVar(vtype=GRB.CONTINUOUS, name="C", lb=0.0) # Cash needed up front  - allocation money
A = m.addVars(range(T), vtype=GRB.CONTINUOUS, name="A", lb=0.0) # Cash Available in time T

#Constraint 1
# Cash allocation in year 0
m.addConstr(A[0] == C - gp.quicksum(COSTS[b] * I[b] for b in B), "AvailableCash[0]")

#Constraint 2
# Available cash at beginning of every year t
for t in range(1, T):
    # Income from bond coupons in year t
    bond_income = gp.quicksum(INCOMES[b, t] * I[b] for b in B)
    # Cash available at the start of year t = previous year's cash * (1 + interest rate) + bond income - required payment
    m.addConstr(
        A[t] == (1 + BASE_r) * A[t - 1] + bond_income - payments[t],
        f"AvailableCash[{t}]"
    )
    
# Objective - Minimize the initial cash for year 1(C)
m.setObjective(C, GRB.MINIMIZE)

# Solve the model
m.optimize()

print(f"Optimal objective value Base Case: ${round(m.objVal, 2)}")

print("All Base Case Results:")
get_solution(m)
get_shadow(m)
get_rc(m)


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i5-1235U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 15 rows, 19 columns and 63 nonzeros
Model fingerprint: 0xe16d298d
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+04, 3e+04]
Presolve removed 1 rows and 2 columns
Presolve time: 0.02s
Presolved: 14 rows, 17 columns, 57 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   3.459274e+04   0.000000e+00      0s
       9    1.8676840e+05   0.000000e+00   0.000000e+00      0s

Solved in 9 iterations and 0.03 seconds (0.00 work units)
Optimal objective  1.867683968e+05
Optimal objective value Base Case: $186768.4
All Base Case Results:
Solution Values:
    I[0]: 73.69479810424208
    I[

## Bad Case
The model above assumes a steady interest rate for all time periods $t=1,..., T$, but that is probably not a safe assumption. For example, an economic downturn is often accompanied by a dropping of interest rates. Further, this may be accompanied by an icrease in payments to the pension fund due to early retirements, etc. Develop a LP model in the cell below that uses the interest rates and payments of the `BAD_CASE` shown above. Assume that the interest rate for cash carried from year t to year t+1 is the interest rate for year t. 

## Bad Case Model

In [17]:
payments = BAD_CASE['c']
interest_rates = BAD_CASE['r']

# Create a new model
m1 = gp.Model('bad_case')

# Create variables
I = m1.addVars(B, vtype=GRB.CONTINUOUS, name="I", lb=0.0) # Investment in each bond (shares to buy)
C = m1.addVar(vtype=GRB.CONTINUOUS, name="C", lb=0.0) # Cash needed up front
A = m1.addVars(range(T), vtype=GRB.CONTINUOUS, name="A", lb=0.0) # Cash Available in time T

# Constraint1
# Cash available in year 0 (initial allocation) 
m1.addConstr(A[0] == C - gp.quicksum(COSTS[b] * I[b] for b in B), "AvailableCash[0]")

#Constraint 2
# Cash available at begining for each year t
for t in range(1, T):
    # Income from bond coupons in year t
    bond_income = gp.quicksum(INCOMES[b, t] * I[b] for b in B)
    # Cash available at the start of year t = previous year's cash available* (1 + interest rate) + bond income - pension payment
    m1.addConstr(
        A[t] == (1 + BAD_CASE['r'][t-1]) * A[t - 1] + bond_income - payments[t],
        f"AvailableCash[{t}]"
    )

# Set the objective: Minimize the initial cash required (C)
m1.setObjective(C, GRB.MINIMIZE)

# Solve the model
m1.optimize()

print(f"Optimal objective value Bad Case: ${round(m1.objVal, 2)}")

print("\nBad Case Results:")
get_solution(m1)
get_shadow(m1)
get_rc(m1)


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i5-1235U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 15 rows, 19 columns and 63 nonzeros
Model fingerprint: 0x0a754220
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+04, 3e+04]
Presolve removed 1 rows and 2 columns
Presolve time: 0.01s
Presolved: 14 rows, 17 columns, 57 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   3.845609e+04   0.000000e+00      0s
       8    2.2104836e+05   0.000000e+00   0.000000e+00      0s

Solved in 8 iterations and 0.02 seconds (0.00 work units)
Optimal objective  2.210483619e+05
Optimal objective value Bad Case: $221048.36

Bad Case Results:
Solution Values:
    I[0]: 105.43119162886187
    I[1]: